In [1]:
try:  # import as appropriate for 2.x vs. 3.x
    import tkinter as tk
    import tkinter.messagebox as tkMessageBox
except:
    import Tkinter as tk
    import tkMessageBox
    

In [2]:
class Defender:
    def __init__(self):
        self.width = 20
        self.height = 20
        self.move_delta = 20
        self.id = None
        self.max_fired_bullets = 8
        self.fired_bullets = []

    def install_in(self, canvas): #carre blanc
        if self.id is not None: return
        cx = int(canvas.cget("width"))
        cy = int(canvas.cget("height")) 
        lx = cx / 2 + self.width / 2
        ly = cy - self.height - 10
        self.id = canvas.create_rectangle(lx,ly, lx + self.width ,ly + self.height ,outline='black',fill='white')  
    
    def coord(self, canvas): #Stockage des coordonnés du defender 
        return canvas.coords(self.id)      

    def move_in(self, canvas, deplacement):
        max_canvas = int(canvas.cget("width")) #On récupere la taille du canvas
        coord_x = self.coord(canvas)[0] #On recupere les coordonnés du défender 
        if deplacement < 0:#Si on va vers la gauche
            if coord_x > 0: #Si on ne dépasse pas le canvas 
                canvas.move(self.id, deplacement, 0)#on se déplace 
        else:
            if coord_x + self.width < max_canvas: 
                canvas.move(self.id, deplacement, 0)
    
    def animation_projectil(self, canvas):#On appel move_in pour chaques bullets
        for i in self.fired_bullets:
            i.move_in(canvas)
        
    def fire(self, canvas):
        if len(self.fired_bullets) < self.max_fired_bullets: #si le nb de bullets est < au max de bullets
            Bullet(self).install_in(canvas)#On creer une nouvelle bullet

    def rm_bullet(self, bullet):# Supprime la bullet de la liste
        self.fired_bullets.remove(bullet)

In [3]:
class Alien(object):
    alien_image = None
    alien_image1= None
    
    def __init__(self):
        self.id = None
        self.alive = True
    
    @classmethod
    def get_image(cls):
        if cls.alien_image==None:
            cls.alien_image = tk.PhotoImage(file="img/alien.gif")#On ajoute l'image
        return cls.alien_image
    
    @classmethod
    def get_deathImage(cls): #On reçoit le gif d'une explosion qui va être stocké dans une variable et retourné
        if cls.alien_image1==None:
            cls.alien_image1 = tk.PhotoImage(file="img/explosion.gif")
        return cls.alien_image1

    def install_in(self,canvas, x, y, image, tag):
        self.id = canvas.create_image(x, y, image=image, tags=tag)# On creer l'image
        
    def img_alien_vivant(self, canvas, x, y):
        self.install_in(canvas, x, y, self.get_image(), "Alien_vivant")
        
    def img_alien_dead(self, canvas, x, y):
        self.install_in(canvas, x, y, self.get_deathImage(), "Alien_dead")
        canvas.after(300, canvas.delete, self.id)
    
    def move_in(self, canvas, dx, dy):
        canvas.move(self.id, dx, dy)
        
    def kill_alien(self, canvas, projectile):
        x, y = canvas.coords(self.id)
        canvas.delete(self.id)
        self.img_alien_dead(canvas, x, y)


In [4]:
class Fleet(object):
    def __init__(self):
        self.aliens_lines = 5
        self.aliens_columns = 10
        self.aliens_inner_gap = 20
        self.alien_x_delta = 5
        self.alien_y_delta = 15
        fleet_size = self.aliens_lines * self.aliens_columns
        self.aliens_fleet = [None] * fleet_size
        
    def install_in(self, canvas):
        x=100
        y=100
        h=0
        for i in range(self.aliens_lines):
            for j in range(self.aliens_columns):
                self.alien = Alien() #On crée un alien 
                self.alien.img_alien_vivant(canvas,x, y)
                self.aliens_fleet[h] = self.alien #On l'ajoute dans la liste
                h+=1
                x+=self.aliens_inner_gap + Alien.get_image().width()#On Maj les coordonnés en X
            y+=self.aliens_inner_gap + Alien.get_image().height()#On Maj les coordonnés en Y
            x=100 #On Maj la position en X
    
    def move_in(self, canvas):
        cwidth = int(canvas.cget("width")) #On récupere la taille du canvas
        
        x1, y1, x2, y2 = canvas.bbox("Alien_vivant") #On récupere la taille de la flotte

        deplacement_y=0

        if(self.alien_x_delta>0): #déplacement vers la droite
            if x2>cwidth: #si sa dépasse le canvas
                self.alien_x_delta = self.alien_x_delta*-1 #on change le signe
                deplacement_y = self.alien_y_delta #on déplace les aliennes vers le bas
        else:
            if x1<0:
                self.alien_x_delta = self.alien_x_delta*-1 
                deplacement_y = self.alien_y_delta 
        
        for i in self.aliens_fleet: 
            i.move_in(canvas, self.alien_x_delta, deplacement_y) #Appel de l'animation pour chaque aliens

    def alien_touche(self,canvas,defender):
        bullets = defender.fired_bullets
        for bull in list(bullets):
            x1, y1, x2, y2 = canvas.coords(bull.id)
            touched = canvas.find_overlapping(x1, y1, x2, y2)
            
            for alien in self.aliens_fleet:
                if alien.alive and alien.id in touched:
                    alien.kill_alien(canvas, defender) #On tue un alien
                    bullets.remove(bull) #on suprime le projectil de la liste
                    canvas.delete(bull.id) #On supprime le projectil du canvas
                    self.aliens_fleet.remove(alien) #On supprime l'alien de la liste
    
    def get_width(self):
        return ((73 + self.aliens_inner_gap) * self.aliens_columns + 200)

In [5]:
class Bullet(object):
    def __init__(self, shooter):
        self.radius = 5
        self.color = "red"
        self.speed = 8
        self.id = None
        self.shooter = shooter
    
    def install_in(self, canvas):
        if self.id == None:
            cx=self.shooter.coord(canvas)[0] + self.shooter.width/2 - self.radius
            cy=self.shooter.coord(canvas)[1] - self.radius*2
            self.id = canvas.create_oval(cx, cy,cx + self.radius*2, cy + self.radius*2, fill=self.color)
            self.shooter.fired_bullets.append(self)#On ajoute une bullet à la liste
    

    def move_in(self, canvas):
        coordonnes = canvas.coords(self.id)#On récupere les cooronnés de la bullet
        if coordonnes[1]>=0:#si elle ne dépasse pas le canvas
            dx = 0
            dy = - self.speed
            canvas.move(self.id, dx, dy)#elle se déplace 
        else:
            canvas.delete(self.id)#On la supprime du canvas
            self.shooter.rm_bullet(self)#On la supprime de la liste
        

In [6]:
class Game(object):
    def __init__(self, frame):
        self.frame = frame
        self.fleet = Fleet()
        self.defender = Defender()
        self.height =800
        self.width = self.fleet.get_width()
        self.canvas = tk.Canvas(self.frame, width=self.width, height=self.height, background="black")
        self.canvas.pack(side="top", fill="both", expand=True)

        self.defender.install_in(self.canvas)
        self.fleet.install_in(self.canvas)
        self.frame.winfo_toplevel().bind("<Key>", self.keypressed)
    
    def keypressed(self, event):
        if event.keysym == 'Left':
            self.defender.move_in(self.canvas, -self.defender.move_delta)
        elif event.keysym == 'Right':
            self.defender.move_in(self.canvas, self.defender.move_delta)
        elif event.keysym == 'space':
            self.defender.fire(self.canvas)
    
    def start_animation(self):
        self.frame.after(30, self.animation) 

    def animation(self): # Lance les animations 
        self.fleet.move_in(self.canvas)
        self.defender.animation_projectil(self.canvas)
        self.fleet.alien_touche(self.canvas, self.defender)
        
        resultat = self.resultat()
        if resultat == -1:
            self.frame.after(30, self.animation) #Si on est en jeu, on appel recurcivement la fonction animation
        elif resultat == 1:
            self.canvas.create_text(self.width/2,self.height/2,text="Bravo ! Vous avez gagné !",font=("Helvetica",30), fill="red")
        else:
            self.canvas.create_text(self.width/2,self.height/2,text="Vous avez perdu...",font=("Helvetica",30), fill="red")

    

    def resultat(self): 
        l = len(self.fleet.aliens_fleet) #On recupere la taille de la liste
        retour = -1
        
        if (l) == 0: #si on gagne 
            retour = 1
        else:
            x1, y1, x2, y2 = self.canvas.bbox("Alien_vivant") #On récupere la taille de la flotte 
            if(y2 > self.height): #si les aliens on gagné
                retour = 0
        return retour

In [7]:
import tkinter as tk
class SpaceInvaders(object):
    
    
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Space Invaders")
        self.frame = tk.Frame(self.root)
        self.frame.pack(side="top", fill="both", expand=True)
        self.game = Game(self.frame)
    
    def play(self):
        self.game.start_animation()
        self.root.mainloop()
        
        
SpaceInvaders().play()